In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import pymysql
import os
import re

from tqdm import tqdm
from transformers import *
from langdetect import detect

2023-04-26 06:08:26.861734: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/envs/mixmind/lib/python3.8/site-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(


In [2]:
import sys
sys.path.append('/home/wnsrb8560/MixMind/mm_bert/korean')

from kor_emotion_analysis import *
from kor_preprocessing import isKorean_percent

loading file vocab.txt from cache at bert_ckpt/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at bert_ckpt/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/tokenizer_config.json
loading configuration file config.json from cache at bert_ckpt/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,


In [3]:
import sys
sys.path.append('/home/wnsrb8560/MixMind/mm_bert/english') #

from eng_emotion_analysis import *

loading configuration file config.json from cache at /home/wnsrb8560/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at /home/wnsrb8560/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/vocab.txt
l

In [4]:
sentiment_model_kor = TFBertClassifier_KOR(model_name='bert-base-multilingual-cased', dir_path='bert_ckpt')
sentiment_model_eng = TFBertClassifier_ENG(model_name='bert-base-cased')

2023-04-25 04:55:21.238025: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-25 04:55:21.596007: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-25 04:55:21.596332: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-25 04:55:21.598568: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operation

In [5]:
# 가중치(한글)
opt = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps = 2344*4, warmup_proportion=0.1, min_lr=1e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
sentiment_model_kor.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

/opt/conda/envs/mixmind/lib/python3.8/site-packages/tensorflow_addons/optimizers/rectified_adam.py:121: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [6]:
# 가중치(영어), this learning rate is for bert model , taken from huggingface website
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=5e-05, epsilon=1e-08,decay=0.01,clipnorm=1.0)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True)
metric = tf.keras.metrics.CategoricalAccuracy('balanced_accuracy'),
sentiment_model_eng.compile(optimizer = optimizer,loss = loss, metrics = metric)

In [7]:
sentiment_model_kor.load_weights('/home/wnsrb8560/MixMind/mm_bert/korean/korean_binary_sentiment_weights/korean_binary_sentiment_weights') #
sentiment_model_eng.load_weights('/home/wnsrb8560/MixMind/mm_bert/english/new_multi_sentiment_weights/new_multi_sentiment_weights') #

In [8]:
# 한글 사전 분류 위한 vocab
vocab_path = '/home/wnsrb8560/MixMind/mm_bert/korean/vocab_9class_500.csv'
stopwords_path = '/home/wnsrb8560/MixMind/mm_bert/korean/stopwords.txt'

vocab = load_vocab(vocab_path, stopwords_path)

In [3]:
# DB에서 리뷰 export
# import pymysql

# db = pymysql.connect(host='34.64.62.157', port=3306, user='', passwd='', dataset='mixmind_webservice', charset='utf8')

# cur = db.cursor()
# query = "Select id, review from mixmind_musicreview"

# review_df_from_db = pd.read_sql(query, db)

review_df_from_db = pd.read_csv('/home/wnsrb8560/MixMind/mm_bert/hiphop_youtube.csv', index_col=None)
#review_df_from_db = review_df_from_db.drop('review_num', axis=1)

In [4]:
review_df_from_db

,youtube_ID,review
0,r6WbbU_lLCA,['Sheesh fam this hits more aggressively than ...
1,azaZt7eccnc,['지코는 진짜 곡을 중독성 있게 엄청 잘 만드는데 그렇다고 중독성만 있는 게 아니...
2,POe9SOEKotk,['BLACKPINK IS THE REVOLUTION SEMPRE SERA LEND...
3,RFMi3v0TXP8,['Pink Venom Out Now https youtu be gQlMMD8auM...
4,dp8fgSykQZQ,['사람들이 듣기 좋은 랩을 하는것도 능력인걸 어쩌겠어 이영지가 하는건 랩이 아니라...
...,...,...
3996,i1VNtZnnTBA,['진짜 초고수들이 칼갈고 와서 일합씩 가르고 가는 기분이네. 비트고 가사고 뭐고 ...
3997,yUDzmKWdD5g,['음악은 결국 꾸준히 하는 사람이 성공합니다 .. 계속 만들어본 사람이 본인이 뭘...
3998,xwhh2GwHUgQ,"['와 이거 제 추억 서린 곡이에요 아 너무 좋아라 가슴이 저릿저릿', '언제 들어..."
3999,bum4aj7goXo,['DON MALIK PAID IN SEOUL 21. 10. 19. TUE. 6PM...


In [11]:
# 리뷰 전처리
for idx, row in enumerate(review_df_from_db.itertuples()):
    temp = list(set(row.review.lstrip('[').rstrip(']').split(',')))
    remove_set = {"'YouTube에서 로그아웃한 상태입니다'", " 'YouTube에서 로그아웃한 상태입니다'", " '로그인하면 동영상에 좋아요를 표시하고 댓글을 달거나 구독할 수 있습니다 '", "'로그인하면 동영상에 좋아요를 표시하고 댓글을 달거나 구독할 수 있습니다 '"}
    clear_temp = [i for i in temp if i not in remove_set]

    review_df_from_db.loc[idx, 'review'] = clear_temp

review_df_from_db

,youtube_ID,review
0,-_bdhLPf1OU,"[ '노래 진짜 잘 하시네요 가사가 귀에 속속들어옵니다', '노래너무 잘한다 이노..."
1,kSzraUekkNE,"[ '최고', '영웅이가 있어 올해도 행복직진 사랑해요', '나에게 힘을주는 노..."
2,Znpnf1HUmQ4,"[ '역시 임영웅 너무 깔껌 단백 최고', '좋은노래가 동닙니다', '그대 사랑..."
3,_yv99sys0oc,"[ '안성훈 님 이노래가 자꾸들어도 또 듣고싶네요 명곡을만들었네요', '안성훈 님..."
4,yaeG5iPP414,"[ '울 이찬원 가수님 풍등 노래 너무 좋아요 계속 승승장구하세요 응원합니다', ..."
...,...,...
3996,zV3nLgfxtfg,"[ '최고', '노래 너무 좋네요', '신지님 노래 너무 좋아요 그리고 슬퍼요'..."
3997,aM0IATEj8lk,[ '장민호 사랑의콜센타 장녹수 감동보이스에 잘하셔두 너무잘하셔요 장녹수 최고최고 ...
3998,W0PCzg8fMzs,"[ '잘하네 요', 'KBS 방송이라 보는분이 별로 없내 ㅋㅋㅋ', '만봉꾼 두..."
3999,nanfnaROHMc,"[ '박인희 박우철', '박우철 가수님 최고중의 최고이십니다. 항상 응원하고 있으..."


In [12]:
# 빈 벡터 데이터 프레임 생성
#predict_emotion = pd.DataFrame(index=range(0,0), columns=['id', 'love', 'joy', 'passion', 'happiness', 'sadness', 'anger', 'loneliness', 'longing', 'fear', 'surprise'])
#predict_emotion.to_csv('dance_predict_emotion.csv', index=False)


In [14]:
# 진행중이던 데이터프레임 불러오기
predict_emotion = pd.read_csv('trot_predict_emotion.csv', index_col=None)
predict_emotion

,id,love,joy,passion,happiness,sadness,anger,loneliness,longing,fear,surprise
0,0.0,0.56487,0.12060,0.06311,0.05661,0.02632,0.02570,0.01033,0.01932,0.00634,0.10680
1,1.0,0.49556,0.21778,0.08924,0.06675,0.04702,0.03239,0.02005,0.01942,0.00632,0.00546
2,2.0,0.50269,0.18707,0.09221,0.09208,0.04480,0.03798,0.01866,0.01826,0.00625,0.00000
3,3.0,0.55686,0.20450,0.08214,0.05793,0.03526,0.03256,0.01540,0.01194,0.00342,0.00000
4,4.0,0.57178,0.17224,0.08700,0.06664,0.03495,0.03182,0.01271,0.01725,0.00561,0.00000
...,...,...,...,...,...,...,...,...,...,...,...
3996,3996.0,0.41390,0.18873,0.09094,0.10050,0.05683,0.04639,0.02401,0.03014,0.00915,0.03941
3997,3997.0,0.61764,0.19651,0.07302,0.03450,0.03236,0.02113,0.01173,0.00981,0.00330,0.00000
3998,3998.0,0.47287,0.21248,0.09112,0.06391,0.04482,0.05079,0.02724,0.02049,0.01628,0.00000
3999,3999.0,0.55984,0.18595,0.07743,0.06337,0.04302,0.03233,0.01545,0.01660,0.00601,0.00000


In [ ]:
# predict_emotion.to_csv('predict_emotion.csv', index=False)

In [13]:
# 리뷰 구분 / 예측

for idx, row in zip(tqdm(range(len(predict_emotion), len(review_df_from_db))), review_df_from_db[len(predict_emotion):].itertuples()):
    temp_dict = {'love' : 0, 'fun': 0, 'enthusiasm' : 0, 'happiness' : 0, 'sadness' : 0, 'anger' : 0, 'loneliness' : 0, 'longing': 0, 'fear' : 0, 'surprise': 0}
    if row.review:
        for review in row.review:
            if isKorean_percent(review, 0.1) == 0: # 한글 10% 이상 있어야 한글로 취급, 아니면 알파벳으로 인식 
                try:
                    if detect(review) == 'en': # 알파벳 인식해도 스페인어나 이런게 섞인 경우가 있음. 영어로 인식하면 감정 처리.
                        eng_emotion = eng_emotion_predict(re.sub('[^a-zA-Z0-9]+', '', review), sentiment_model_eng)
                        for key1, val1 in eng_emotion.items():
                            temp_dict[key1] += val1
                except: # 스페인어 등은 패스
                    continue

            else:
                kor_emotion = hybrid_emotion_export_persent(re.sub('[^가-힣ㄱ-ㅎ0-9]+', '', review), sentiment_model_kor, vocab)
                # 영어, 한글 섞인 리뷰들 중 한글로 취급된 애들은 아예 한글만 보기 위해 알파벳 빼줌.
                for key2, val2 in kor_emotion.items():
                    temp_dict[key2] += val2
        
        if sum(temp_dict.values()) != 0:
            predict_emotion.loc[idx, 'id'] =  idx
            predict_emotion.loc[idx, 'love'] = np.round((temp_dict['love'] / sum(temp_dict.values())), 5)
            predict_emotion.loc[idx, 'joy'] = np.round((temp_dict['fun'] / sum(temp_dict.values())), 5)
            predict_emotion.loc[idx, 'passion'] = np.round((temp_dict['enthusiasm'] / sum(temp_dict.values())), 5)
            predict_emotion.loc[idx, 'happiness'] = np.round((temp_dict['happiness'] / sum(temp_dict.values())), 5)
            predict_emotion.loc[idx, 'sadness'] = np.round((temp_dict['sadness'] / sum(temp_dict.values())), 5)
            predict_emotion.loc[idx, 'anger'] = np.round((temp_dict['anger'] / sum(temp_dict.values())), 5)
            predict_emotion.loc[idx, 'loneliness'] = np.round((temp_dict['loneliness'] / sum(temp_dict.values())), 5)
            predict_emotion.loc[idx, 'longing'] = np.round((temp_dict['longing'] / sum(temp_dict.values())), 5)
            predict_emotion.loc[idx, 'fear'] = np.round((temp_dict['fear'] / sum(temp_dict.values())), 5)
            predict_emotion.loc[idx, 'surprise'] = np.round((temp_dict['surprise'] / sum(temp_dict.values())), 5)
            predict_emotion.to_csv('trot_predict_emotion.csv', index=False)
        
        else: # 리뷰에 스페인어 등만 있는, 영양가 없는 리뷰만 긁어서 temp_dict 합 값이 0이 될수 있음.
            predict_emotion[idx, 'id'] = idx
            predict_emotion.to_csv('trot_predict_emotion.csv', index=False)
            continue

    else: # 리뷰 빈 칸인 경우
        predict_emotion[idx, 'id'] = idx
        predict_emotion.to_csv('trot_predict_emotion.csv', index=False)
        continue

  0%|          | 0/49 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/envs/mixmind/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1/1 [==============================] - 0s 39ms/step


  2%|▏         | 1/49 [00:29<23:46, 29.72s/it]

1/1 [==============================] - 0s 42ms/step


  4%|▍         | 2/49 [00:44<16:30, 21.07s/it]

...

 96%|█████████▌| 47/49 [09:57<00:40, 20.22s/it]

1/1 [==============================] - 0s 42ms/step


 98%|█████████▊| 48/49 [10:21<00:21, 21.22s/it]

1/1 [==============================] - 0s 40ms/step


100%|██████████| 49/49 [10:40<00:00, 13.07s/it]


In [ ]:
# 위 결과 db로
connection = pymysql.connect(host='34.64.62.157', user='TeamMixMind', port=3306, password="730402", db='mixmind_webservice', charset='utf8mb4', use_unicode=True, cursorclass=pymysql.cursors.DictCursor)
cur = connection.cursor()

for row in tqdm(predict_emotion.itertuples()):
    
    sql = """INSERT INTO mixmind_musicemotion (love, joy, passion, happiness, sadness, anger, loneliness, longing, fear, surprise, musicId_id) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    cur.execute(sql, (str(row.love), str(row.joy), str(row.passion), str(row.happiness), str(row.sadness), str(row.anger), str(row.loneliness), str(row.longing), str(row.fear), str(row.surprise), int(row.id)))
    connection.commit()

connection.close()